In [22]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split as split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score as score
from sklearn.model_selection import cross_val_score as cross

In [23]:
df = pd.read_csv('All_CSV/train.csv', usecols=['Age', 'Fare', 'Survived'])

In [24]:
df.isnull().sum()

Survived      0
Age         177
Fare          0
dtype: int64

In [25]:
train_x, test_x, train_y, test_y = split(df.drop(columns=['Survived']), df['Survived'], test_size = 0.2)

# without kbins

In [26]:
transformer = ColumnTransformer(transformers=[
    ('fill', SimpleImputer(), ['Age'])
], remainder = 'passthrough')

In [27]:
new_train_x = transformer.fit_transform(train_x)
new_test_x = transformer.transform(test_x)

In [28]:
t_train_x = pd.DataFrame(new_train_x, columns=['Age', 'Fare'])
t_test_x = pd.DataFrame(new_test_x, columns=['Age', 'Fare'])

In [29]:
decision = DecisionTreeClassifier()

In [30]:
decision.fit(t_train_x, train_y)

DecisionTreeClassifier()

In [31]:
pred = decision.predict(t_test_x)

In [32]:
score(pred, test_y)

0.6368715083798883

In [33]:
np.mean(cross(DecisionTreeClassifier(),t_train_x, train_y ,cv=100,scoring='accuracy'))

0.6364285714285713

# With Kbins

In [34]:
transformer2 = ColumnTransformer( transformers=[
    ('fill', SimpleImputer(), ['Age']),
    ('discreeter1', KBinsDiscretizer(n_bins = 15, encode = 'ordinal', strategy = 'equal_width'), ['Age']),
    ('discreeter2', KBinsDiscretizer(n_bins = 15, encode = 'ordinal', strategy = 'equal_width'), ['Fare'])

], remainder = 'passthrough')

# quantile
# uniform
# equal_width

In [35]:
new2_train_x = transformer.fit_transform(train_x)
new2_test_x = transformer.transform(test_x)

In [36]:
t2_train_x = pd.DataFrame(new2_train_x, columns=['Age', 'Fare'])
t2_test_x = pd.DataFrame(new2_test_x, columns=['Age', 'Fare'])

In [37]:
decision2 = DecisionTreeClassifier()

In [38]:
decision2.fit(t_train_x, train_y)

DecisionTreeClassifier()

In [39]:
pred2 = decision2.predict(t_test_x)

In [40]:
score(pred2, test_y)

0.6312849162011173

In [41]:
np.mean(cross(DecisionTreeClassifier(),t2_train_x, train_y ,cv=100,scoring='accuracy'))

0.6303571428571428

# Binarisarion

In [42]:
train = pd.read_csv('All_CSV/train.csv',usecols = ['Age', 'Fare', 'SibSp', 'Parch', 'Survived'])

In [43]:
train.head()

,Survived,Age,SibSp,Parch,Fare
0,0,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,1,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,0,35.0,0,0,8.0500


In [44]:
train['Family'] = train['SibSp']+train['Parch']

In [45]:
train = train.drop(columns=['SibSp', 'Parch'])

In [46]:
from sklearn.preprocessing import Binarizer

In [47]:
train_x, test_x, train_y, test_y = split(train.drop(columns=['Survived']), train['Survived'], test_size = 0.2)

In [48]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 875 to 863
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Age     574 non-null    float64
 1   Fare    712 non-null    float64
 2   Family  712 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 22.2 KB


In [49]:
transformer = ColumnTransformer(transformers = [
    ('fill', SimpleImputer(), ['Age']),
    ('polarisation1',Binarizer(), ['Family']),
    ('polarisation2', Binarizer(), ['Fare']),
    ('polarisation3', Binarizer(), ['Age'])
], remainder = 'passthrough')

In [50]:
new_train_x = transformer.fit_transform(train_x)
new_test_x = transformer.transform(test_x)

In [51]:
t_train_x = pd.DataFrame(new_train_x, columns=['Age', 'Fare', 'Survived'])
t_test_x = pd.DataFrame(new_test_x, columns=['Age', 'Fare', 'Survived'])

In [52]:
decision = DecisionTreeClassifier()

In [53]:
decision.fit(t_train_x, train_y)

DecisionTreeClassifier()

In [54]:
pred = decision.predict(t_test_x)

In [55]:
score(pred, test_y)

0.6368715083798883

In [56]:
np.mean(cross(decision, t_train_x, train_y, cv=100, scoring = 'accuracy'))

0.664285714285714